In [ ]:
import numpy as np
import pandas as pd 
import string

import nltk
from nltk.corpus import stopwords

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/amazon-fine-food-reviews/Reviews.csv")
df.head()

In [ ]:
df.describe()

In [ ]:
df.Score.value_counts()

In [ ]:
sns.countplot(x='Score', data=df, palette='RdBu')
plt.xlabel('Score (Rating)')
plt.show()

5-star reviews constitute a large proportion (64%) of all reviews. The next most prevalent rating is 4-stars(14%), followed by 1-star (9%), 3-star (8%), and finally 2-star reviews (5%).

In [ ]:
temp_df = df[['UserId','HelpfulnessNumerator','HelpfulnessDenominator', 'Summary', 'Text','Score']].copy()
temp_df["Sentiment"] = temp_df["Score"].apply(lambda score: "positive" if score > 3 else ("negative" if score < 3 else "not defined"))
temp_df["Usefulness"] = (temp_df["HelpfulnessNumerator"] / temp_df["HelpfulnessDenominator"]).apply\
(lambda n: ">75%" if n > 0.75 else ("<25%" if n < 0.25 else ("25-75%" if n >= 0.25 and n <= 0.75 else "useless")))
temp_df.loc[temp_df.HelpfulnessDenominator == 0, 'Usefulness'] = "useless"
temp_df.head()

In [ ]:
sns.countplot(x='Sentiment', order=["positive", "negative"], data=temp_df, palette='RdBu')
plt.xlabel('Sentiment')
plt.show()

In [ ]:
pos = temp_df.loc[temp_df['Sentiment'] == 'positive']
pos = pos[:1000]
neg = temp_df.loc[temp_df['Sentiment'] == 'negative']
neg = neg[:1000]

In [ ]:
def create_Word_Corpus(temp):
    words_corpus = ''
    for val in temp["Summary"]:
        text = str(val).lower()
        #text = text.translate(trantab)
        tokens = nltk.word_tokenize(text)
        tokens = [word for word in tokens if word not in stopwords.words('english')]
        for words in tokens:
            words_corpus = words_corpus + words + ' '
    return words_corpus

In [ ]:
pos_wordcloud = WordCloud(width=900, height=500).generate(create_Word_Corpus(pos))
neg_wordcloud = WordCloud(width=900, height=500).generate(create_Word_Corpus(neg))

In [ ]:
def plot_Cloud(wordCloud):
    plt.figure( figsize=(20,10), facecolor='w')
    plt.imshow(wordCloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()

In [ ]:
plot_Cloud(pos_wordcloud)

In [ ]:
plot_Cloud(neg_wordcloud)

In [ ]:
temp_df.Usefulness.value_counts()

In [ ]:
sns.countplot(x='Usefulness', order=['useless', '>75%', '25-75%', '<25%'], data=temp_df)
plt.xlabel('Usefulness')
plt.show()

In [ ]:
sns.countplot(x='Sentiment', hue='Usefulness', order=["positive", "negative"],hue_order=['>75%', '25-75%', '<25%'], data=temp_df)
plt.xlabel('Sentiment')
plt.show()

In [ ]:
temp_df["text_word_count"] = temp_df["Text"].apply(lambda text: len(text.split()))
sns.boxplot(x='Score',y='text_word_count', data=temp_df, showfliers=False)
plt.show()

In [ ]:
sns.violinplot(x='Usefulness', y='text_word_count', order=[">75%", "<25%"], data=temp_df)
plt.ylim(-50, 400)
plt.show()

In [ ]:
x = temp_df.UserId.value_counts()
x.to_dict()
temp_df["reviewer_freq"] = temp_df["UserId"].apply(lambda counts: "Frequent (>50 reviews)" if x[counts]>50 else "Not Frequent (1-50)")

In [ ]:
ax = sns.countplot(x='Score', hue='reviewer_freq', data=temp_df)
ax.set_xlabel('Score (Rating)')
plt.show()

In [ ]:
y = temp_df[temp_df.reviewer_freq=="Frequent (>50 reviews)"].Score.value_counts()
z = temp_df[temp_df.reviewer_freq=="Not Frequent (1-50)"].Score.value_counts()
tot_y = y.sum()
y = (y/tot_y)*100
tot_z = z.sum()
z = (z/tot_z)*100

In [ ]:
ax1 = plt.subplot(131)
y.plot(kind="bar",ax=ax1)
plt.xlabel("Score")
plt.ylabel("Percentage")
plt.title("Frequent (>50 reviews) Distribution")

ax2 = plt.subplot(133)
z.plot(kind="bar",ax=ax2)
plt.xlabel("Score")
plt.ylabel("Percentage")
plt.title("Not Frequent (1-50) Distribution")
plt.show()

In [ ]:
sns.countplot(x='Usefulness', order=['useless', '>75%', '25-75%', '<25%'], hue='reviewer_freq', data=temp_df)
plt.xlabel('Helpfulness')
plt.show()

In [ ]:
sns.violinplot(x='reviewer_freq', y='text_word_count', data=temp_df, palette='RdBu')
plt.xlabel('Frequency of Reviewer')
plt.ylim(-50, 400)
plt.show()

* Positive reviews are very common.
* Positive reviews are shorter.
* Longer reviews are more helpful.
* Despite being more common and shorter, positive reviews are found more helpful.
* Frequent reviewers are more discerning in their ratings, write longer reviews, and write more helpful reviews